# सामग्री सूची
- [गाइडेंसको बारेमा](../../../../code/01.Introduce)
- [सेटअप](../../../../code/01.Introduce)
- [अप्रतिबन्धित सिर्जना](../../../../code/01.Introduce)
- [फी ३ को लागि बोल्दै](../../../../code/01.Introduce)
- [रेगुलर एक्सप्रेशन](../../../../code/01.Introduce)
- [चयन](../../../../code/01.Introduce)
- [विचारको श्रृंखला](../../../../code/01.Introduce)
- [JSON सिर्जना](../../../../code/01.Introduce)
- [HTML सिर्जना](../../../../code/01.Introduce)


# Guidance को बारेमा  
Guidance कुनै पनि भाषा मोडेल (LM) को नतिजालाई नियन्त्रण गर्नको लागि प्रमाणित ओपन-सोर्स Python लाइब्रेरी हो। एक API कलको माध्यमबाट, तपाईं Python मा मोडेलले पालना गर्नुपर्ने ठ्याक्कै प्रोग्रामेटिक बाधा(हरू) व्यक्त गर्न सक्नुहुन्छ र JSON, Python, HTML, SQL, वा प्रयोगको आवश्यकता अनुसारको संरचित नतिजा उत्पन्न गर्न सक्नुहुन्छ।  

Guidance परम्परागत prompting प्रविधिहरूबाट फरक छ। यो मोडेललाई टोकन बाई टोकन inference लेयरमा निर्देशित गरेर बाधाहरू लागू गर्दछ, जसले उच्च गुणस्तरको नतिजा उत्पादन गर्दछ र अत्यधिक संरचित परिदृश्यहरूमा प्रयोग गर्दा लागत र विलम्बता ३०–५०% सम्म घटाउँछ।  

Guidance को बारेमा थप जान्नको लागि, [GitHub मा सार्वजनिक रिपोजिटरी](https://github.com/guidance-ai/guidance) भ्रमण गर्नुहोस् वा Microsoft Build मा भएको [Guidance Breakout Session](https://www.youtube.com/watch?v=qXMNPVVlCMs) हेर्नुहोस्।  


# सेटअप
1. Guidance लाई `pip install guidance --pre` प्रयोग गरेर इन्स्टल गर्नुहोस्
2. Azure मा Phi 3.5 mini endpoint डिप्लोय गर्नुहोस्। यसका लागि https://ai.azure.com/explore/models/Phi-3.5-mini-instruct/version/2/registry/azureml मा जानुहोस् र "Deploy" बटनमा क्लिक गर्नुहोस्
3. आफ्नो endpoint को API key लाई `AZURE_PHI3_KEY` नामको environment variable मा राख्नुहोस् र URL लाई `AZURE_PHI3_URL` नामको environment variable मा राख्नुहोस्


In [ ]:
from guidance import gen, select, regex, user, assistant, system, json
from guidance.models import AzureGuidance
from json import loads as load_json_str
import os

phi3_url = os.getenv("AZURE_PHI3_URL")
phi3_api_key = os.getenv("AZURE_PHI3_KEY")
phi3_lm = AzureGuidance(f"{phi3_url}/guidance#auth={phi3_api_key}")

# Or, load from HuggingFace to run locally
# from guidance.models import Transformers
# phi3_lm = Transformers("microsoft/Phi-3-mini-4k-instruct")

# स्वतन्त्र सिर्जना
पाठलाई कुनै पनि सीमाबन्धन बिना `gen()` फङ्सन प्रयोग गरेर सिर्जना गर्न सकिन्छ। यो Guidance बिना मोडेल प्रयोग गरेजस्तै हो।

## च्याट स्वरूप
धेरै च्याट मोडेलहरू जस्तै, Phi-3 ले प्रयोगकर्ता र सहायक बीचको सन्देशलाई विशेष स्वरूपमा अपेक्षा गर्दछ। Guidance ले Phi-3 को च्याट टेम्प्लेटलाई समर्थन गर्दछ र तपाईंको लागि च्याट स्वरूप व्यवस्थापन गर्नेछ। च्याटको पालो सिर्जना गर्न, कुराकानीको प्रत्येक भागलाई `with user()` वा `with assistant()` ब्लकमा राख्नुहोस्। प्रणाली सन्देश सेट गर्न `with system()` ब्लक प्रयोग गर्न सकिन्छ।


In [22]:
lm = phi3_lm
with system():
    lm += "You are a helpful assistant. You have a cranky yet entertaining temperament."
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += gen(temperature=0.8, max_tokens=100)

## टोकन बचत  
धेरै संरचित परिदृश्यहरूमा, Guidance ले टोकनहरू छोड्न सक्छ र केवल आवश्यक टोकनहरू उत्पन्न गर्न सक्छ, जसले प्रदर्शन सुधार गर्दछ, दक्षता बढाउँछ र API लागत बचत गर्दछ। उत्पन्न टोकनहरू यस नोटबुकमा हाइलाइट गरिएको पृष्ठभूमिसँग देखाइन्छ। बाध्य टोकनहरू बिना हाइलाइट देखाइन्छ र इनपुट टोकनहरूको जस्तै लागत लाग्छ, जुन आउटपुट टोकनहरूको लागतको एक तिहाइ अनुमान गरिएको छ।  

*नोट:* बिना कुनै सीमाबन्धनको साथ पहिलो उदाहरणले कुनै टोकन बाध्य गर्न सकेन किनकि हामीले कुनै सीमाहरू प्रदान गरेका थिएनौं।  


# फाई ३ को लागि बोल्दै  
मार्गदर्शनको साथ, तपाईं सजिलै मोडेलको प्रतिक्रियामा पाठ समावेश गर्न सक्नुहुन्छ। यदि तपाईं मोडेलको उत्पादनलाई विशेष दिशामा मार्गदर्शन गर्न चाहनुहुन्छ भने यो उपयोगी हुन सक्छ।


In [5]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + gen(temperature=0.8, max_tokens=50)

# रेगुलर एक्सप्रेशन प्रयोग गरेर सीमित गर्नु
अघिल्लो उदाहरणमा, Phi 3 ले `Canberra` भन्ने प्रश्नको उत्तर दिएपछि थप व्याख्या गरेको थियो। मोडेलको आउटपुटलाई ठ्याक्कै एक शब्दमा सीमित गर्न, रेगुलर एक्सप्रेशन प्रयोग गर्न सकिन्छ।


In [6]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + regex("[A-Z][a-z]+")

# धेरै विकल्पहरूबाट चयन गर्दै
जब केही सम्भावित विकल्पहरू थाहा हुन्छ, तपाईं `select()` फङ्सन प्रयोग गरेर मोडेललाई विकल्पहरूको सूचीबाट चयन गर्न सक्नुहुन्छ।


In [23]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + select(["Washington", "Canberra", "Sydney", "Melbourne"])

`select()` प्रयोग गर्दा, केवल टोकन `Can` उत्पन्न भयो। किनभने `Canberra` मात्र विकल्प हो जसले सम्भवतः प्रतिक्रिया पूरा गर्न सक्छ, बाँकी टोकनहरू बाध्य गरियो।


# विचारको श्रृंखला
विचारको श्रृंखला एउटा प्रविधि हो जसले मोडेलको उत्पादनको गुणस्तर सुधार गर्न मद्दत गर्न सक्छ, यसलाई समस्या चरणबद्ध रूपमा प्रक्रिया गर्न प्रेरित गरेर। सामान्यतया, अन्तिम उत्तरमा पुग्नका लागि धेरै पटक प्रम्प्ट आवश्यक हुन्छ। पहिलो, मोडेललाई चरणबद्ध रूपमा सोच्न निर्देशन दिनुहोस्। त्यसपछि, मोडेललाई अन्तिम उत्तर दिन पुनः प्रम्प्ट गर्नुहोस्। मानक च्याट इनफरेन्स API हरूसँग, यसले २ API कलहरू लिन्छ, र मोडेलले उत्पन्न गरेको "विचारको श्रृंखला" दुई पटक चार्ज हुन्छ – एक पटक मोडेलले यसलाई उत्पादन गर्दा आउटपुट टोकनको रूपमा, र अर्को पटक दोस्रो कलको लागि इनपुट टोकनको रूपमा। Guidance को प्रयोगले, सम्पूर्ण बहु-चरण प्रक्रिया एकल API कलको भागको रूपमा प्रक्रिया र चार्ज गरिन्छ, लागत र विलम्बता घटाउँदै।


In [8]:
gsm8k_question = "Mark has a garden with flowers. He planted plants of three different colors in it. Ten of them are yellow, and there are 80% more of those in purple. There are only 25% as many green flowers as there are yellow and purple flowers. How many flowers does Mark have in his garden?"
lm = phi3_lm
with user():
    lm += gsm8k_question
with assistant():
    lm += "Let's think step by step. " + gen(temperature=0.8, max_tokens=500)
    # Prompt for the final answer, which should be a number. Store the output in an "answer" variable.
    lm += "\nTherefore, the final answer is: " + regex(r"\d+", name="answer")

print(f"Final answer: {lm['answer']}")

Final answer: 35


# JSON निर्माण
Guidance प्रयोग गरेर JSON स्किमाको वा pydantic मोडेलको अनुरूप JSON निर्माण सुनिश्चित गर्न सकिन्छ, जस्तै यहाँ देखाइएको प्रयोगकर्ता प्रोफाइल स्किमा।


In [16]:
user_json_schema = load_json_str("""{
  "$schema": "http://json-schema.org/draft-07/schema#",
  "title": "User Profile",
  "type": "object",
  "properties": {
    "username": {
      "type": "string"
    },
    "age": {
      "type": "integer"
    },
    "email": {
      "type": "string"
    }
  },
  "additionalProperties": false
}
""")

lm = phi3_lm
with user():
    lm += "Generate a JSON object for a user profile. The profile should include a username, age, email, and nothing more."

with assistant():
    lm += json(schema=user_json_schema, temperature=1.0)

In [19]:
from pydantic import BaseModel

class UserProfile(BaseModel):
    username: str
    age: int
    email: str


lm = phi3_lm
with user():
    lm += "Generate a JSON object for a user profile. The profile should include a username, age, email, and nothing more."

with assistant():
    lm += json(schema=UserProfile, temperature=1.0)

## HTML निर्माण

Guidance प्रयोग गरेर कोड निर्माण गर्न र प्रोग्रामिङ भाषाको साङ्गठनिक आवश्यकताहरू पालना गर्न सकिन्छ। यस खण्डमा, हामी साधारण HTML वेबपेजहरू लेख्नका लागि सानो Guidance कार्यक्रम बनाउनेछौं।

हामी वेबपेजलाई साना भागहरूमा विभाजन गर्नेछौं, प्रत्येकको आफ्नै Guidance function हुनेछ। यी सबैलाई अन्तिम function मा संयोजन गरेर HTML वेबपेज निर्माण गरिनेछ।  
त्यसपछि हामी यस function लाई Guidance-enabled model मा Azure AI मार्फत चलाउनेछौं।

*Note:* यो पूर्ण रूपमा विकसित HTML निर्माणकर्ता हुने छैन; उद्देश्य भनेको तपाईंको व्यक्तिगत आवश्यकताहरूका लागि संरचित आउटपुट कसरी निर्माण गर्न सकिन्छ भन्ने देखाउनु हो।

हामी Guidance बाट आवश्यक सामग्री आयात गरेर सुरु गर्छौं:


In [ ]:
from guidance import guidance
from guidance.library import (
    zero_or_more,
    any_char_but,
    select,
    capture,
    with_temperature,
)
from guidance.models import Model

HTML वेबपेजहरू अत्यधिक संरचित हुन्छन्, र हामी ती पृष्ठका भागहरूलाई Guidance प्रयोग गरेर 'जबरजस्ती' गर्नेछौं।  
जब हामी मोडेलबाट स्पष्ट रूपमा पाठ चाहन्छौं, हामीले सुनिश्चित गर्नुपर्छ कि यसले कुनै पनि कुरा समावेश गर्दैन जुन ट्याग हुन सक्छ - अर्थात्, हामीले '<' र '>' अक्षरहरूलाई बाहिर राख्नुपर्छ।


In [ ]:
@guidance(stateless=True)
def _gen_text(lm: Model):
    return lm + zero_or_more(any_char_but(["<", ">"]))

त्यसपछि हामी यो फङ्सन प्रयोग गरेर मनपर्ने HTML ट्यागभित्र पाठ उत्पन्न गर्न सक्छौं:


In [ ]:
@guidance(stateless=True)
def _gen_text_in_tag(lm: Model, tag: str):
    lm += f"<{tag}>"
    lm += _gen_text()
    lm += f"</{tag}>"
    return lm

पृष्ठ शीर्षक सिर्जना गरौं।  
यसको भागको रूपमा, हामीले पृष्ठको शीर्षक तयार गर्न आवश्यक छ:


In [ ]:
@guidance(stateless=True)
def _gen_header(lm: Model):
    lm += "<head>\n"
    lm += _gen_text_in_tag("title") + "\n"
    lm += "</head>\n"
    return lm

HTML पृष्ठको मुख्य भाग शीर्षकहरू र अनुच्छेदहरूले भरिनेछ।  
हामी प्रत्येक कार्य गर्नको लागि एउटा फङ्सन परिभाषित गर्न सक्छौं:


In [ ]:
@guidance(stateless=True)
def _gen_heading(lm: Model):
    lm += select(
        options=[_gen_text_in_tag("h1"), _gen_text_in_tag("h2"), _gen_text_in_tag("h3")]
    )
    lm += "\n"
    return lm

@guidance(stateless=True)
def _gen_para(lm: Model):
    lm += _gen_text_in_tag("p")
    lm += "\n"
    return lm

अब, HTML को मुख्य भाग परिभाषित गर्ने कार्य।

यसले `select()` लाई `recurse=True` को साथ प्रयोग गरेर धेरै शीर्षकहरू र अनुच्छेदहरू सिर्जना गर्दछ:


In [ ]:
@guidance(stateless=True)
def _gen_body(lm: Model):
    lm += "<body>\n"
    lm += select(options=[_gen_heading(), _gen_para()], recurse=True)
    lm += "</body>\n"
    return lm

अब, हामी पूर्ण HTML पृष्ठ निर्माण गर्ने कार्यमा आउँछौं।  
हामी HTML सुरु गर्ने ट्याग थप्छौं, त्यसपछि हेडर निर्माण गर्छौं, त्यसपछि बडी, र अन्त्यमा HTML समाप्त गर्ने ट्याग जोड्छौं:


In [ ]:
@guidance(stateless=True)
def _gen_html(lm: Model):
    lm += "<html>\n"
    lm += _gen_header()
    lm += _gen_body()
    lm += "</html>\n"
    return lm

हामी एक प्रयोगकर्ता-मित्रवत् आवरण प्रदान गर्छौं, जसले हामीलाई निम्न कार्यहरू गर्न अनुमति दिनेछ:
- जेनेरेशनको तापक्रम सेट गर्नुहोस्
- मोडेल वस्तुबाट उत्पन्न पृष्ठ कब्जा गर्नुहोस्


In [ ]:
@guidance(stateless=True)
def html(
    lm,
    name: str | None = None,
    *,
    temperature: float = 0.0,
):
    return lm + capture(
        with_temperature(_gen_html(), temperature=temperature),
        name=name,
    )

In [ ]:
lm = phi3_lm

lm += "Create a web page about your life story. Split your uplifting tale into multiple paragraphs with headings:\n"
lm += html(name="html_text", temperature=0.7)

त्यसपछि हामी परिणामलाई फाइलमा लेख्न सक्छौं:


In [ ]:
with open('./sample_page.html', 'w') as html_file:
    html_file.write(lm["html_text"])

र [परिणाम हेर्नुहोस्](../../../../code/01.Introduce/sample_page.html)।



---

**अस्वीकरण**:  
यो दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) प्रयोग गरेर अनुवाद गरिएको हो। हामी शुद्धताको लागि प्रयास गर्छौं, तर कृपया ध्यान दिनुहोस् कि स्वचालित अनुवादमा त्रुटिहरू वा अशुद्धताहरू हुन सक्छ। यसको मूल भाषा मा रहेको मूल दस्तावेज़लाई आधिकारिक स्रोत मानिनुपर्छ। महत्वपूर्ण जानकारीको लागि, व्यावसायिक मानव अनुवाद सिफारिस गरिन्छ। यस अनुवादको प्रयोगबाट उत्पन्न हुने कुनै पनि गलतफहमी वा गलत व्याख्याको लागि हामी जिम्मेवार हुने छैनौं।
